In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.neighbors import NearestNeighbors

In [2]:
metric = 'cosine'
user = 1
n = 10

In [3]:
data = pd.read_json('../data/ratings_data.json', orient = 'split')
n_users = data['userId'].unique().shape[0]
n_items = data['movieId'].unique().shape[0]
users = data['userId'].unique()
movies = data['movieId'].unique()

In [4]:
data_matrix = pd.DataFrame(np.zeros((n_users, n_items)), columns = movies, index = users)
for line in data.itertuples():
    data_matrix.at[line[4], line[1]] = line[2]

In [5]:
user_similarity = 1 - pairwise_distances(data_matrix, metric=metric)

In [6]:
def predict(ratings, similarity):
    mean_user_rating = ratings.mean(axis=1)
    ratings_diff = (ratings - mean_user_rating[:, np.newaxis])
    pred = mean_user_rating[:, np.newaxis] + similarity.dot(ratings_diff) / np.array([np.abs(similarity).sum(axis=1)]).T

    return pred

In [7]:
user_prediction = pd.DataFrame(predict(data_matrix.values, user_similarity), columns = movies, index = users)

In [8]:
def recommend(ratings, user, n = 10):
    user_movies = pd.DataFrame(data_matrix.loc[user])
    user_movies.columns = ['rating']
    viewed_movies = user_movies[user_movies['rating'] > 0].index
    predicted_ratings = pd.DataFrame(ratings.loc[user])
    predicted_ratings.columns = ['rating']
    recommended_movies = predicted_ratings.drop(viewed_movies).sort_values(['rating'], ascending=[0]).head(n).index.values

    return recommended_movies

In [9]:
recommend(user_prediction, user, n)

array([ 260,  296, 1196,  318, 1198,  593,  858, 1270,  608,  356],
      dtype=int64)